In [1]:
import zipfile, io, json, os
from kaggle.api.kaggle_api_extended import KaggleApi
import pandas as pd

In [2]:
files = [f for f in os.listdir('.') if os.path.isfile(f)]
if 'basketball.zip' not in files and 'basketball.sqlite' not in files:
        api = KaggleApi()
        #config_dir = os.getcwd()
        #config_file = 'kaggle.json'
        #api.config = os.path.join(config_dir, config_file)
        api.authenticate()
        api.dataset_download_files('wyattowalsh/basketball') #downloads to cwd
elif 'basketball.zip' in files and 'basketball.sqlite' not in files:
    with zipfile.ZipFile('basketball.zip', 'r') as zip_ref:
        zip_ref.extractall()
    os.remove('daily_execution_pipeline.yml')
    os.remove('monthly_execution_pipeline.yml')

if 'basketball.sqlite' in files:
    %load_ext sql
    %sql sqlite:///basketball.sqlite

In [4]:
season_start=2018

In [3]:
%%sql
    SELECT DISTINCT TEAM_ID_HOME
    FROM Game
    WHERE Season >= 2019

 * sqlite:///basketball.sqlite
Done.


TEAM_ID_HOME
1610612746
1610612761
1610612755
1610612759
1610612756
1610612751
1610612753
1610612748
1610612757
1610612762


In [5]:
%%sql team_stats_1610612746 << WITH home_games AS ( SELECT ROW_NUMBER() OVER(ORDER BY GAME_DATE) AS Game_Num_Home,
        TEAM_ID_HOME AS TEAM_ID, TEAM_ABBREVIATION_HOME AS TEAM_ABBREVIATION,
        AVG(FGM_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGM_HOME_avg,
        AVG(FGA_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGA_HOME_avg,
        AVG(FG3M_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_HOME_avg,
        AVG(FG3M_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_HOME_avg,
        AVG(FTM_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTM_HOME_avg,
        AVG(FTA_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTA_HOME_avg,
        AVG(OREB_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS OREB_HOME_avg,
        AVG(DREB_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS DREB_HOME_avg,
        AVG(AST_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS AST_HOME_avg,
        AVG(STL_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS STL_HOME_avg,
        AVG(TOV_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS TOV_HOME_avg,
        AVG(PF_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PF_HOME_avg,
        AVG(PTS_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PTS_HOME_avg,
        CASE
            WHEN WL_HOME = 'W'
            THEN 1
            ELSE 0
        END AS home_win
FROM Game
WHERE Season >= {season_start} AND TEAM_ID_HOME = 1610612746
),
away_games AS (
    SELECT ROW_NUMBER() OVER(ORDER BY GAME_DATE) AS Game_Num_Away,
        AVG(FGM_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGM_AWAY_avg,
        AVG(FGA_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGA_AWAY_avg,
        AVG(FG3M_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_AWAY_avg,
        AVG(FG3M_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_AWAY_avg,
        AVG(FTM_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTM_AWAY_avg,
        AVG(FTA_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTA_AWAY_avg,
        AVG(OREB_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS OREB_AWAY_avg,
        AVG(DREB_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS DREB_AWAY_avg,
        AVG(AST_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS AST_AWAY_avg,
        AVG(STL_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS STL_AWAY_avg,
        AVG(TOV_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS TOV_AWAY_avg,
        AVG(PF_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PF_AWAY_avg,
        AVG(PTS_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PTS_AWAY_avg,
        CASE
            WHEN WL_AWAY = 'W'
            THEN 1
            ELSE 0
        END AS away_win
    FROM Game
    WHERE Season >= {season_start} AND TEAM_ID_AWAY = 1610612746
)
SELECT * FROM home_games h, away_games a
WHERE h.Game_Num_Home = a.Game_Num_Away

 * sqlite:///basketball.sqlite
Done.
Returning data to local variable team_stats_1610612746


In [6]:
team_stats_1610612746.DataFrame().shape

(113, 32)

In [7]:
%%sql team_stats_1610612761 << WITH home_games AS ( SELECT ROW_NUMBER() OVER(ORDER BY GAME_DATE) AS Game_Num_Home,
        TEAM_ID_HOME AS TEAM_ID, TEAM_ABBREVIATION_HOME AS TEAM_ABBREVIATION,
        AVG(FGM_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGM_HOME_avg,
        AVG(FGA_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGA_HOME_avg,
        AVG(FG3M_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_HOME_avg,
        AVG(FG3M_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_HOME_avg,
        AVG(FTM_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTM_HOME_avg,
        AVG(FTA_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTA_HOME_avg,
        AVG(OREB_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS OREB_HOME_avg,
        AVG(DREB_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS DREB_HOME_avg,
        AVG(AST_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS AST_HOME_avg,
        AVG(STL_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS STL_HOME_avg,
        AVG(TOV_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS TOV_HOME_avg,
        AVG(PF_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PF_HOME_avg,
        AVG(PTS_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PTS_HOME_avg,
        CASE
            WHEN WL_HOME = 'W'
            THEN 1
            ELSE 0
        END AS home_win
FROM Game
WHERE Season >= {season_start} AND TEAM_ID_HOME = 1610612761
),
away_games AS (
    SELECT ROW_NUMBER() OVER(ORDER BY GAME_DATE) AS Game_Num_Away,
        AVG(FGM_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGM_AWAY_avg,
        AVG(FGA_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGA_AWAY_avg,
        AVG(FG3M_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_AWAY_avg,
        AVG(FG3M_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_AWAY_avg,
        AVG(FTM_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTM_AWAY_avg,
        AVG(FTA_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTA_AWAY_avg,
        AVG(OREB_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS OREB_AWAY_avg,
        AVG(DREB_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS DREB_AWAY_avg,
        AVG(AST_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS AST_AWAY_avg,
        AVG(STL_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS STL_AWAY_avg,
        AVG(TOV_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS TOV_AWAY_avg,
        AVG(PF_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PF_AWAY_avg,
        AVG(PTS_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PTS_AWAY_avg,
        CASE
            WHEN WL_AWAY = 'W'
            THEN 1
            ELSE 0
        END AS away_win
    FROM Game
    WHERE Season >= {season_start} AND TEAM_ID_AWAY = 1610612761
)
SELECT * FROM home_games h, away_games a
WHERE h.Game_Num_Home = a.Game_Num_Away

 * sqlite:///basketball.sqlite
Done.
Returning data to local variable team_stats_1610612761


In [8]:
%%sql team_stats_1610612755 << WITH home_games AS ( SELECT ROW_NUMBER() OVER(ORDER BY GAME_DATE) AS Game_Num_Home,
        TEAM_ID_HOME AS TEAM_ID, TEAM_ABBREVIATION_HOME AS TEAM_ABBREVIATION,
        AVG(FGM_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGM_HOME_avg,
        AVG(FGA_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGA_HOME_avg,
        AVG(FG3M_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_HOME_avg,
        AVG(FG3M_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_HOME_avg,
        AVG(FTM_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTM_HOME_avg,
        AVG(FTA_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTA_HOME_avg,
        AVG(OREB_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS OREB_HOME_avg,
        AVG(DREB_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS DREB_HOME_avg,
        AVG(AST_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS AST_HOME_avg,
        AVG(STL_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS STL_HOME_avg,
        AVG(TOV_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS TOV_HOME_avg,
        AVG(PF_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PF_HOME_avg,
        AVG(PTS_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PTS_HOME_avg,
        CASE
            WHEN WL_HOME = 'W'
            THEN 1
            ELSE 0
        END AS home_win
FROM Game
WHERE Season >= {season_start} AND TEAM_ID_HOME = 1610612755
),
away_games AS (
    SELECT ROW_NUMBER() OVER(ORDER BY GAME_DATE) AS Game_Num_Away,
        AVG(FGM_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGM_AWAY_avg,
        AVG(FGA_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGA_AWAY_avg,
        AVG(FG3M_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_AWAY_avg,
        AVG(FG3M_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_AWAY_avg,
        AVG(FTM_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTM_AWAY_avg,
        AVG(FTA_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTA_AWAY_avg,
        AVG(OREB_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS OREB_AWAY_avg,
        AVG(DREB_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS DREB_AWAY_avg,
        AVG(AST_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS AST_AWAY_avg,
        AVG(STL_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS STL_AWAY_avg,
        AVG(TOV_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS TOV_AWAY_avg,
        AVG(PF_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PF_AWAY_avg,
        AVG(PTS_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PTS_AWAY_avg,
        CASE
            WHEN WL_AWAY = 'W'
            THEN 1
            ELSE 0
        END AS away_win
    FROM Game
    WHERE Season >= {season_start} AND TEAM_ID_AWAY = 1610612755
)
SELECT * FROM home_games h, away_games a
WHERE h.Game_Num_Home = a.Game_Num_Away

 * sqlite:///basketball.sqlite
Done.
Returning data to local variable team_stats_1610612755


In [9]:
%%sql team_stats_1610612759 << WITH home_games AS ( SELECT ROW_NUMBER() OVER(ORDER BY GAME_DATE) AS Game_Num_Home,
        TEAM_ID_HOME AS TEAM_ID, TEAM_ABBREVIATION_HOME AS TEAM_ABBREVIATION,
        AVG(FGM_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGM_HOME_avg,
        AVG(FGA_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGA_HOME_avg,
        AVG(FG3M_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_HOME_avg,
        AVG(FG3M_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_HOME_avg,
        AVG(FTM_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTM_HOME_avg,
        AVG(FTA_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTA_HOME_avg,
        AVG(OREB_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS OREB_HOME_avg,
        AVG(DREB_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS DREB_HOME_avg,
        AVG(AST_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS AST_HOME_avg,
        AVG(STL_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS STL_HOME_avg,
        AVG(TOV_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS TOV_HOME_avg,
        AVG(PF_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PF_HOME_avg,
        AVG(PTS_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PTS_HOME_avg,
        CASE
            WHEN WL_HOME = 'W'
            THEN 1
            ELSE 0
        END AS home_win
FROM Game
WHERE Season >= {season_start} AND TEAM_ID_HOME = 1610612759
),
away_games AS (
    SELECT ROW_NUMBER() OVER(ORDER BY GAME_DATE) AS Game_Num_Away,
        AVG(FGM_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGM_AWAY_avg,
        AVG(FGA_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGA_AWAY_avg,
        AVG(FG3M_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_AWAY_avg,
        AVG(FG3M_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_AWAY_avg,
        AVG(FTM_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTM_AWAY_avg,
        AVG(FTA_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTA_AWAY_avg,
        AVG(OREB_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS OREB_AWAY_avg,
        AVG(DREB_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS DREB_AWAY_avg,
        AVG(AST_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS AST_AWAY_avg,
        AVG(STL_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS STL_AWAY_avg,
        AVG(TOV_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS TOV_AWAY_avg,
        AVG(PF_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PF_AWAY_avg,
        AVG(PTS_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PTS_AWAY_avg,
        CASE
            WHEN WL_AWAY = 'W'
            THEN 1
            ELSE 0
        END AS away_win
    FROM Game
    WHERE Season >= {season_start} AND TEAM_ID_AWAY = 1610612759
)
SELECT * FROM home_games h, away_games a
WHERE h.Game_Num_Home = a.Game_Num_Away

 * sqlite:///basketball.sqlite
Done.
Returning data to local variable team_stats_1610612759


In [10]:
%%sql team_stats_1610612756 << WITH home_games AS ( SELECT ROW_NUMBER() OVER(ORDER BY GAME_DATE) AS Game_Num_Home,
        TEAM_ID_HOME AS TEAM_ID, TEAM_ABBREVIATION_HOME AS TEAM_ABBREVIATION,
        AVG(FGM_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGM_HOME_avg,
        AVG(FGA_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGA_HOME_avg,
        AVG(FG3M_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_HOME_avg,
        AVG(FG3M_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_HOME_avg,
        AVG(FTM_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTM_HOME_avg,
        AVG(FTA_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTA_HOME_avg,
        AVG(OREB_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS OREB_HOME_avg,
        AVG(DREB_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS DREB_HOME_avg,
        AVG(AST_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS AST_HOME_avg,
        AVG(STL_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS STL_HOME_avg,
        AVG(TOV_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS TOV_HOME_avg,
        AVG(PF_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PF_HOME_avg,
        AVG(PTS_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PTS_HOME_avg,
        CASE
            WHEN WL_HOME = 'W'
            THEN 1
            ELSE 0
        END AS home_win
FROM Game
WHERE Season >= {season_start} AND TEAM_ID_HOME = 1610612756
),
away_games AS (
    SELECT ROW_NUMBER() OVER(ORDER BY GAME_DATE) AS Game_Num_Away,
        AVG(FGM_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGM_AWAY_avg,
        AVG(FGA_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGA_AWAY_avg,
        AVG(FG3M_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_AWAY_avg,
        AVG(FG3M_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_AWAY_avg,
        AVG(FTM_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTM_AWAY_avg,
        AVG(FTA_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTA_AWAY_avg,
        AVG(OREB_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS OREB_AWAY_avg,
        AVG(DREB_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS DREB_AWAY_avg,
        AVG(AST_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS AST_AWAY_avg,
        AVG(STL_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS STL_AWAY_avg,
        AVG(TOV_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS TOV_AWAY_avg,
        AVG(PF_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PF_AWAY_avg,
        AVG(PTS_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PTS_AWAY_avg,
        CASE
            WHEN WL_AWAY = 'W'
            THEN 1
            ELSE 0
        END AS away_win
    FROM Game
    WHERE Season >= {season_start} AND TEAM_ID_AWAY = 1610612756
)
SELECT * FROM home_games h, away_games a
WHERE h.Game_Num_Home = a.Game_Num_Away

 * sqlite:///basketball.sqlite
Done.
Returning data to local variable team_stats_1610612756


In [11]:
%%sql team_stats_1610612751 << WITH home_games AS ( SELECT ROW_NUMBER() OVER(ORDER BY GAME_DATE) AS Game_Num_Home,
        TEAM_ID_HOME AS TEAM_ID, TEAM_ABBREVIATION_HOME AS TEAM_ABBREVIATION,
        AVG(FGM_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGM_HOME_avg,
        AVG(FGA_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGA_HOME_avg,
        AVG(FG3M_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_HOME_avg,
        AVG(FG3M_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_HOME_avg,
        AVG(FTM_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTM_HOME_avg,
        AVG(FTA_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTA_HOME_avg,
        AVG(OREB_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS OREB_HOME_avg,
        AVG(DREB_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS DREB_HOME_avg,
        AVG(AST_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS AST_HOME_avg,
        AVG(STL_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS STL_HOME_avg,
        AVG(TOV_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS TOV_HOME_avg,
        AVG(PF_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PF_HOME_avg,
        AVG(PTS_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PTS_HOME_avg,
        CASE
            WHEN WL_HOME = 'W'
            THEN 1
            ELSE 0
        END AS home_win
FROM Game
WHERE Season >= {season_start} AND TEAM_ID_HOME = 1610612751
),
away_games AS (
    SELECT ROW_NUMBER() OVER(ORDER BY GAME_DATE) AS Game_Num_Away,
        AVG(FGM_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGM_AWAY_avg,
        AVG(FGA_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGA_AWAY_avg,
        AVG(FG3M_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_AWAY_avg,
        AVG(FG3M_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_AWAY_avg,
        AVG(FTM_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTM_AWAY_avg,
        AVG(FTA_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTA_AWAY_avg,
        AVG(OREB_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS OREB_AWAY_avg,
        AVG(DREB_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS DREB_AWAY_avg,
        AVG(AST_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS AST_AWAY_avg,
        AVG(STL_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS STL_AWAY_avg,
        AVG(TOV_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS TOV_AWAY_avg,
        AVG(PF_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PF_AWAY_avg,
        AVG(PTS_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PTS_AWAY_avg,
        CASE
            WHEN WL_AWAY = 'W'
            THEN 1
            ELSE 0
        END AS away_win
    FROM Game
    WHERE Season >= {season_start} AND TEAM_ID_AWAY = 1610612751
)
SELECT * FROM home_games h, away_games a
WHERE h.Game_Num_Home = a.Game_Num_Away

 * sqlite:///basketball.sqlite
Done.
Returning data to local variable team_stats_1610612751


In [12]:
%%sql team_stats_1610612753 << WITH home_games AS ( SELECT ROW_NUMBER() OVER(ORDER BY GAME_DATE) AS Game_Num_Home,
        TEAM_ID_HOME AS TEAM_ID, TEAM_ABBREVIATION_HOME AS TEAM_ABBREVIATION,
        AVG(FGM_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGM_HOME_avg,
        AVG(FGA_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGA_HOME_avg,
        AVG(FG3M_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_HOME_avg,
        AVG(FG3M_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_HOME_avg,
        AVG(FTM_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTM_HOME_avg,
        AVG(FTA_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTA_HOME_avg,
        AVG(OREB_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS OREB_HOME_avg,
        AVG(DREB_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS DREB_HOME_avg,
        AVG(AST_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS AST_HOME_avg,
        AVG(STL_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS STL_HOME_avg,
        AVG(TOV_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS TOV_HOME_avg,
        AVG(PF_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PF_HOME_avg,
        AVG(PTS_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PTS_HOME_avg,
        CASE
            WHEN WL_HOME = 'W'
            THEN 1
            ELSE 0
        END AS home_win
FROM Game
WHERE Season >= {season_start} AND TEAM_ID_HOME = 1610612753
),
away_games AS (
    SELECT ROW_NUMBER() OVER(ORDER BY GAME_DATE) AS Game_Num_Away,
        AVG(FGM_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGM_AWAY_avg,
        AVG(FGA_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGA_AWAY_avg,
        AVG(FG3M_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_AWAY_avg,
        AVG(FG3M_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_AWAY_avg,
        AVG(FTM_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTM_AWAY_avg,
        AVG(FTA_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTA_AWAY_avg,
        AVG(OREB_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS OREB_AWAY_avg,
        AVG(DREB_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS DREB_AWAY_avg,
        AVG(AST_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS AST_AWAY_avg,
        AVG(STL_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS STL_AWAY_avg,
        AVG(TOV_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS TOV_AWAY_avg,
        AVG(PF_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PF_AWAY_avg,
        AVG(PTS_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PTS_AWAY_avg,
        CASE
            WHEN WL_AWAY = 'W'
            THEN 1
            ELSE 0
        END AS away_win
    FROM Game
    WHERE Season >= {season_start} AND TEAM_ID_AWAY = 1610612753
)
SELECT * FROM home_games h, away_games a
WHERE h.Game_Num_Home = a.Game_Num_Away

 * sqlite:///basketball.sqlite
Done.
Returning data to local variable team_stats_1610612753


In [13]:
%%sql team_stats_1610612748 << WITH home_games AS ( SELECT ROW_NUMBER() OVER(ORDER BY GAME_DATE) AS Game_Num_Home,
        TEAM_ID_HOME AS TEAM_ID, TEAM_ABBREVIATION_HOME AS TEAM_ABBREVIATION,
        AVG(FGM_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGM_HOME_avg,
        AVG(FGA_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGA_HOME_avg,
        AVG(FG3M_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_HOME_avg,
        AVG(FG3M_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_HOME_avg,
        AVG(FTM_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTM_HOME_avg,
        AVG(FTA_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTA_HOME_avg,
        AVG(OREB_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS OREB_HOME_avg,
        AVG(DREB_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS DREB_HOME_avg,
        AVG(AST_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS AST_HOME_avg,
        AVG(STL_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS STL_HOME_avg,
        AVG(TOV_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS TOV_HOME_avg,
        AVG(PF_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PF_HOME_avg,
        AVG(PTS_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PTS_HOME_avg,
        CASE
            WHEN WL_HOME = 'W'
            THEN 1
            ELSE 0
        END AS home_win
FROM Game
WHERE Season >= {season_start} AND TEAM_ID_HOME = 1610612748
),
away_games AS (
    SELECT ROW_NUMBER() OVER(ORDER BY GAME_DATE) AS Game_Num_Away,
        AVG(FGM_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGM_AWAY_avg,
        AVG(FGA_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGA_AWAY_avg,
        AVG(FG3M_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_AWAY_avg,
        AVG(FG3M_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_AWAY_avg,
        AVG(FTM_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTM_AWAY_avg,
        AVG(FTA_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTA_AWAY_avg,
        AVG(OREB_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS OREB_AWAY_avg,
        AVG(DREB_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS DREB_AWAY_avg,
        AVG(AST_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS AST_AWAY_avg,
        AVG(STL_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS STL_AWAY_avg,
        AVG(TOV_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS TOV_AWAY_avg,
        AVG(PF_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PF_AWAY_avg,
        AVG(PTS_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PTS_AWAY_avg,
        CASE
            WHEN WL_AWAY = 'W'
            THEN 1
            ELSE 0
        END AS away_win
    FROM Game
    WHERE Season >= {season_start} AND TEAM_ID_AWAY = 1610612748
)
SELECT * FROM home_games h, away_games a
WHERE h.Game_Num_Home = a.Game_Num_Away

 * sqlite:///basketball.sqlite
Done.
Returning data to local variable team_stats_1610612748


In [14]:
%%sql team_stats_1610612757 << WITH home_games AS ( SELECT ROW_NUMBER() OVER(ORDER BY GAME_DATE) AS Game_Num_Home,
        TEAM_ID_HOME AS TEAM_ID, TEAM_ABBREVIATION_HOME AS TEAM_ABBREVIATION,
        AVG(FGM_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGM_HOME_avg,
        AVG(FGA_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGA_HOME_avg,
        AVG(FG3M_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_HOME_avg,
        AVG(FG3M_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_HOME_avg,
        AVG(FTM_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTM_HOME_avg,
        AVG(FTA_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTA_HOME_avg,
        AVG(OREB_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS OREB_HOME_avg,
        AVG(DREB_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS DREB_HOME_avg,
        AVG(AST_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS AST_HOME_avg,
        AVG(STL_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS STL_HOME_avg,
        AVG(TOV_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS TOV_HOME_avg,
        AVG(PF_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PF_HOME_avg,
        AVG(PTS_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PTS_HOME_avg,
        CASE
            WHEN WL_HOME = 'W'
            THEN 1
            ELSE 0
        END AS home_win
FROM Game
WHERE Season >= {season_start} AND TEAM_ID_HOME = 1610612757
),
away_games AS (
    SELECT ROW_NUMBER() OVER(ORDER BY GAME_DATE) AS Game_Num_Away,
        AVG(FGM_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGM_AWAY_avg,
        AVG(FGA_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGA_AWAY_avg,
        AVG(FG3M_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_AWAY_avg,
        AVG(FG3M_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_AWAY_avg,
        AVG(FTM_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTM_AWAY_avg,
        AVG(FTA_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTA_AWAY_avg,
        AVG(OREB_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS OREB_AWAY_avg,
        AVG(DREB_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS DREB_AWAY_avg,
        AVG(AST_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS AST_AWAY_avg,
        AVG(STL_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS STL_AWAY_avg,
        AVG(TOV_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS TOV_AWAY_avg,
        AVG(PF_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PF_AWAY_avg,
        AVG(PTS_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PTS_AWAY_avg,
        CASE
            WHEN WL_AWAY = 'W'
            THEN 1
            ELSE 0
        END AS away_win
    FROM Game
    WHERE Season >= {season_start} AND TEAM_ID_AWAY = 1610612757
)
SELECT * FROM home_games h, away_games a
WHERE h.Game_Num_Home = a.Game_Num_Away

 * sqlite:///basketball.sqlite
Done.
Returning data to local variable team_stats_1610612757


In [15]:
%%sql team_stats_1610612762 << WITH home_games AS ( SELECT ROW_NUMBER() OVER(ORDER BY GAME_DATE) AS Game_Num_Home,
        TEAM_ID_HOME AS TEAM_ID, TEAM_ABBREVIATION_HOME AS TEAM_ABBREVIATION,
        AVG(FGM_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGM_HOME_avg,
        AVG(FGA_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGA_HOME_avg,
        AVG(FG3M_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_HOME_avg,
        AVG(FG3M_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_HOME_avg,
        AVG(FTM_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTM_HOME_avg,
        AVG(FTA_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTA_HOME_avg,
        AVG(OREB_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS OREB_HOME_avg,
        AVG(DREB_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS DREB_HOME_avg,
        AVG(AST_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS AST_HOME_avg,
        AVG(STL_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS STL_HOME_avg,
        AVG(TOV_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS TOV_HOME_avg,
        AVG(PF_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PF_HOME_avg,
        AVG(PTS_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PTS_HOME_avg,
        CASE
            WHEN WL_HOME = 'W'
            THEN 1
            ELSE 0
        END AS home_win
FROM Game
WHERE Season >= {season_start} AND TEAM_ID_HOME = 1610612762
),
away_games AS (
    SELECT ROW_NUMBER() OVER(ORDER BY GAME_DATE) AS Game_Num_Away,
        AVG(FGM_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGM_AWAY_avg,
        AVG(FGA_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGA_AWAY_avg,
        AVG(FG3M_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_AWAY_avg,
        AVG(FG3M_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_AWAY_avg,
        AVG(FTM_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTM_AWAY_avg,
        AVG(FTA_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTA_AWAY_avg,
        AVG(OREB_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS OREB_AWAY_avg,
        AVG(DREB_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS DREB_AWAY_avg,
        AVG(AST_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS AST_AWAY_avg,
        AVG(STL_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS STL_AWAY_avg,
        AVG(TOV_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS TOV_AWAY_avg,
        AVG(PF_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PF_AWAY_avg,
        AVG(PTS_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PTS_AWAY_avg,
        CASE
            WHEN WL_AWAY = 'W'
            THEN 1
            ELSE 0
        END AS away_win
    FROM Game
    WHERE Season >= {season_start} AND TEAM_ID_AWAY = 1610612762
)
SELECT * FROM home_games h, away_games a
WHERE h.Game_Num_Home = a.Game_Num_Away

 * sqlite:///basketball.sqlite
Done.
Returning data to local variable team_stats_1610612762


In [16]:
%%sql team_stats_1610612754 << WITH home_games AS ( SELECT ROW_NUMBER() OVER(ORDER BY GAME_DATE) AS Game_Num_Home,
        TEAM_ID_HOME AS TEAM_ID, TEAM_ABBREVIATION_HOME AS TEAM_ABBREVIATION,
        AVG(FGM_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGM_HOME_avg,
        AVG(FGA_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGA_HOME_avg,
        AVG(FG3M_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_HOME_avg,
        AVG(FG3M_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_HOME_avg,
        AVG(FTM_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTM_HOME_avg,
        AVG(FTA_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTA_HOME_avg,
        AVG(OREB_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS OREB_HOME_avg,
        AVG(DREB_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS DREB_HOME_avg,
        AVG(AST_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS AST_HOME_avg,
        AVG(STL_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS STL_HOME_avg,
        AVG(TOV_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS TOV_HOME_avg,
        AVG(PF_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PF_HOME_avg,
        AVG(PTS_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PTS_HOME_avg,
        CASE
            WHEN WL_HOME = 'W'
            THEN 1
            ELSE 0
        END AS home_win
FROM Game
WHERE Season >= {season_start} AND TEAM_ID_HOME = 1610612754
),
away_games AS (
    SELECT ROW_NUMBER() OVER(ORDER BY GAME_DATE) AS Game_Num_Away,
        AVG(FGM_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGM_AWAY_avg,
        AVG(FGA_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGA_AWAY_avg,
        AVG(FG3M_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_AWAY_avg,
        AVG(FG3M_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_AWAY_avg,
        AVG(FTM_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTM_AWAY_avg,
        AVG(FTA_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTA_AWAY_avg,
        AVG(OREB_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS OREB_AWAY_avg,
        AVG(DREB_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS DREB_AWAY_avg,
        AVG(AST_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS AST_AWAY_avg,
        AVG(STL_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS STL_AWAY_avg,
        AVG(TOV_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS TOV_AWAY_avg,
        AVG(PF_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PF_AWAY_avg,
        AVG(PTS_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PTS_AWAY_avg,
        CASE
            WHEN WL_AWAY = 'W'
            THEN 1
            ELSE 0
        END AS away_win
    FROM Game
    WHERE Season >= {season_start} AND TEAM_ID_AWAY = 1610612754
)
SELECT * FROM home_games h, away_games a
WHERE h.Game_Num_Home = a.Game_Num_Away

 * sqlite:///basketball.sqlite
Done.
Returning data to local variable team_stats_1610612754


In [17]:
%%sql team_stats_1610612742 << WITH home_games AS ( SELECT ROW_NUMBER() OVER(ORDER BY GAME_DATE) AS Game_Num_Home,
        TEAM_ID_HOME AS TEAM_ID, TEAM_ABBREVIATION_HOME AS TEAM_ABBREVIATION,
        AVG(FGM_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGM_HOME_avg,
        AVG(FGA_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGA_HOME_avg,
        AVG(FG3M_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_HOME_avg,
        AVG(FG3M_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_HOME_avg,
        AVG(FTM_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTM_HOME_avg,
        AVG(FTA_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTA_HOME_avg,
        AVG(OREB_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS OREB_HOME_avg,
        AVG(DREB_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS DREB_HOME_avg,
        AVG(AST_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS AST_HOME_avg,
        AVG(STL_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS STL_HOME_avg,
        AVG(TOV_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS TOV_HOME_avg,
        AVG(PF_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PF_HOME_avg,
        AVG(PTS_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PTS_HOME_avg,
        CASE
            WHEN WL_HOME = 'W'
            THEN 1
            ELSE 0
        END AS home_win
FROM Game
WHERE Season >= {season_start} AND TEAM_ID_HOME = 1610612742
),
away_games AS (
    SELECT ROW_NUMBER() OVER(ORDER BY GAME_DATE) AS Game_Num_Away,
        AVG(FGM_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGM_AWAY_avg,
        AVG(FGA_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGA_AWAY_avg,
        AVG(FG3M_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_AWAY_avg,
        AVG(FG3M_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_AWAY_avg,
        AVG(FTM_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTM_AWAY_avg,
        AVG(FTA_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTA_AWAY_avg,
        AVG(OREB_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS OREB_AWAY_avg,
        AVG(DREB_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS DREB_AWAY_avg,
        AVG(AST_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS AST_AWAY_avg,
        AVG(STL_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS STL_AWAY_avg,
        AVG(TOV_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS TOV_AWAY_avg,
        AVG(PF_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PF_AWAY_avg,
        AVG(PTS_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PTS_AWAY_avg,
        CASE
            WHEN WL_AWAY = 'W'
            THEN 1
            ELSE 0
        END AS away_win
    FROM Game
    WHERE Season >= {season_start} AND TEAM_ID_AWAY = 1610612742
)
SELECT * FROM home_games h, away_games a
WHERE h.Game_Num_Home = a.Game_Num_Away

 * sqlite:///basketball.sqlite
Done.
Returning data to local variable team_stats_1610612742


In [18]:
%%sql team_stats_1610612766 << WITH home_games AS ( SELECT ROW_NUMBER() OVER(ORDER BY GAME_DATE) AS Game_Num_Home,
        TEAM_ID_HOME AS TEAM_ID, TEAM_ABBREVIATION_HOME AS TEAM_ABBREVIATION,
        AVG(FGM_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGM_HOME_avg,
        AVG(FGA_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGA_HOME_avg,
        AVG(FG3M_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_HOME_avg,
        AVG(FG3M_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_HOME_avg,
        AVG(FTM_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTM_HOME_avg,
        AVG(FTA_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTA_HOME_avg,
        AVG(OREB_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS OREB_HOME_avg,
        AVG(DREB_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS DREB_HOME_avg,
        AVG(AST_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS AST_HOME_avg,
        AVG(STL_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS STL_HOME_avg,
        AVG(TOV_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS TOV_HOME_avg,
        AVG(PF_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PF_HOME_avg,
        AVG(PTS_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PTS_HOME_avg,
        CASE
            WHEN WL_HOME = 'W'
            THEN 1
            ELSE 0
        END AS home_win
FROM Game
WHERE Season >= {season_start} AND TEAM_ID_HOME = 1610612766
),
away_games AS (
    SELECT ROW_NUMBER() OVER(ORDER BY GAME_DATE) AS Game_Num_Away,
        AVG(FGM_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGM_AWAY_avg,
        AVG(FGA_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGA_AWAY_avg,
        AVG(FG3M_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_AWAY_avg,
        AVG(FG3M_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_AWAY_avg,
        AVG(FTM_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTM_AWAY_avg,
        AVG(FTA_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTA_AWAY_avg,
        AVG(OREB_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS OREB_AWAY_avg,
        AVG(DREB_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS DREB_AWAY_avg,
        AVG(AST_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS AST_AWAY_avg,
        AVG(STL_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS STL_AWAY_avg,
        AVG(TOV_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS TOV_AWAY_avg,
        AVG(PF_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PF_AWAY_avg,
        AVG(PTS_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PTS_AWAY_avg,
        CASE
            WHEN WL_AWAY = 'W'
            THEN 1
            ELSE 0
        END AS away_win
    FROM Game
    WHERE Season >= {season_start} AND TEAM_ID_AWAY = 1610612766
)
SELECT * FROM home_games h, away_games a
WHERE h.Game_Num_Home = a.Game_Num_Away

 * sqlite:///basketball.sqlite
Done.
Returning data to local variable team_stats_1610612766


In [19]:
%%sql team_stats_1610612744 << WITH home_games AS ( SELECT ROW_NUMBER() OVER(ORDER BY GAME_DATE) AS Game_Num_Home,
        TEAM_ID_HOME AS TEAM_ID, TEAM_ABBREVIATION_HOME AS TEAM_ABBREVIATION,
        AVG(FGM_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGM_HOME_avg,
        AVG(FGA_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGA_HOME_avg,
        AVG(FG3M_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_HOME_avg,
        AVG(FG3M_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_HOME_avg,
        AVG(FTM_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTM_HOME_avg,
        AVG(FTA_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTA_HOME_avg,
        AVG(OREB_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS OREB_HOME_avg,
        AVG(DREB_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS DREB_HOME_avg,
        AVG(AST_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS AST_HOME_avg,
        AVG(STL_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS STL_HOME_avg,
        AVG(TOV_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS TOV_HOME_avg,
        AVG(PF_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PF_HOME_avg,
        AVG(PTS_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PTS_HOME_avg,
        CASE
            WHEN WL_HOME = 'W'
            THEN 1
            ELSE 0
        END AS home_win
FROM Game
WHERE Season >= {season_start} AND TEAM_ID_HOME = 1610612744
),
away_games AS (
    SELECT ROW_NUMBER() OVER(ORDER BY GAME_DATE) AS Game_Num_Away,
        AVG(FGM_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGM_AWAY_avg,
        AVG(FGA_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGA_AWAY_avg,
        AVG(FG3M_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_AWAY_avg,
        AVG(FG3M_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_AWAY_avg,
        AVG(FTM_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTM_AWAY_avg,
        AVG(FTA_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTA_AWAY_avg,
        AVG(OREB_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS OREB_AWAY_avg,
        AVG(DREB_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS DREB_AWAY_avg,
        AVG(AST_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS AST_AWAY_avg,
        AVG(STL_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS STL_AWAY_avg,
        AVG(TOV_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS TOV_AWAY_avg,
        AVG(PF_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PF_AWAY_avg,
        AVG(PTS_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PTS_AWAY_avg,
        CASE
            WHEN WL_AWAY = 'W'
            THEN 1
            ELSE 0
        END AS away_win
    FROM Game
    WHERE Season >= {season_start} AND TEAM_ID_AWAY = 1610612744
)
SELECT * FROM home_games h, away_games a
WHERE h.Game_Num_Home = a.Game_Num_Away

 * sqlite:///basketball.sqlite
Done.
Returning data to local variable team_stats_1610612744


In [20]:
%%sql team_stats_1610612745 << WITH home_games AS ( SELECT ROW_NUMBER() OVER(ORDER BY GAME_DATE) AS Game_Num_Home,
        TEAM_ID_HOME AS TEAM_ID, TEAM_ABBREVIATION_HOME AS TEAM_ABBREVIATION,
        AVG(FGM_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGM_HOME_avg,
        AVG(FGA_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGA_HOME_avg,
        AVG(FG3M_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_HOME_avg,
        AVG(FG3M_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_HOME_avg,
        AVG(FTM_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTM_HOME_avg,
        AVG(FTA_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTA_HOME_avg,
        AVG(OREB_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS OREB_HOME_avg,
        AVG(DREB_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS DREB_HOME_avg,
        AVG(AST_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS AST_HOME_avg,
        AVG(STL_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS STL_HOME_avg,
        AVG(TOV_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS TOV_HOME_avg,
        AVG(PF_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PF_HOME_avg,
        AVG(PTS_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PTS_HOME_avg,
        CASE
            WHEN WL_HOME = 'W'
            THEN 1
            ELSE 0
        END AS home_win
FROM Game
WHERE Season >= {season_start} AND TEAM_ID_HOME = 1610612745
),
away_games AS (
    SELECT ROW_NUMBER() OVER(ORDER BY GAME_DATE) AS Game_Num_Away,
        AVG(FGM_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGM_AWAY_avg,
        AVG(FGA_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGA_AWAY_avg,
        AVG(FG3M_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_AWAY_avg,
        AVG(FG3M_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_AWAY_avg,
        AVG(FTM_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTM_AWAY_avg,
        AVG(FTA_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTA_AWAY_avg,
        AVG(OREB_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS OREB_AWAY_avg,
        AVG(DREB_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS DREB_AWAY_avg,
        AVG(AST_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS AST_AWAY_avg,
        AVG(STL_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS STL_AWAY_avg,
        AVG(TOV_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS TOV_AWAY_avg,
        AVG(PF_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PF_AWAY_avg,
        AVG(PTS_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PTS_AWAY_avg,
        CASE
            WHEN WL_AWAY = 'W'
            THEN 1
            ELSE 0
        END AS away_win
    FROM Game
    WHERE Season >= {season_start} AND TEAM_ID_AWAY = 1610612745
)
SELECT * FROM home_games h, away_games a
WHERE h.Game_Num_Home = a.Game_Num_Away

 * sqlite:///basketball.sqlite
Done.
Returning data to local variable team_stats_1610612745


In [21]:
%%sql team_stats_1610612765 << WITH home_games AS ( SELECT ROW_NUMBER() OVER(ORDER BY GAME_DATE) AS Game_Num_Home,
        TEAM_ID_HOME AS TEAM_ID, TEAM_ABBREVIATION_HOME AS TEAM_ABBREVIATION,
        AVG(FGM_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGM_HOME_avg,
        AVG(FGA_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGA_HOME_avg,
        AVG(FG3M_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_HOME_avg,
        AVG(FG3M_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_HOME_avg,
        AVG(FTM_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTM_HOME_avg,
        AVG(FTA_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTA_HOME_avg,
        AVG(OREB_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS OREB_HOME_avg,
        AVG(DREB_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS DREB_HOME_avg,
        AVG(AST_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS AST_HOME_avg,
        AVG(STL_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS STL_HOME_avg,
        AVG(TOV_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS TOV_HOME_avg,
        AVG(PF_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PF_HOME_avg,
        AVG(PTS_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PTS_HOME_avg,
        CASE
            WHEN WL_HOME = 'W'
            THEN 1
            ELSE 0
        END AS home_win
FROM Game
WHERE Season >= {season_start} AND TEAM_ID_HOME = 1610612765
),
away_games AS (
    SELECT ROW_NUMBER() OVER(ORDER BY GAME_DATE) AS Game_Num_Away,
        AVG(FGM_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGM_AWAY_avg,
        AVG(FGA_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGA_AWAY_avg,
        AVG(FG3M_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_AWAY_avg,
        AVG(FG3M_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_AWAY_avg,
        AVG(FTM_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTM_AWAY_avg,
        AVG(FTA_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTA_AWAY_avg,
        AVG(OREB_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS OREB_AWAY_avg,
        AVG(DREB_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS DREB_AWAY_avg,
        AVG(AST_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS AST_AWAY_avg,
        AVG(STL_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS STL_AWAY_avg,
        AVG(TOV_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS TOV_AWAY_avg,
        AVG(PF_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PF_AWAY_avg,
        AVG(PTS_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PTS_AWAY_avg,
        CASE
            WHEN WL_AWAY = 'W'
            THEN 1
            ELSE 0
        END AS away_win
    FROM Game
    WHERE Season >= {season_start} AND TEAM_ID_AWAY = 1610612765
)
SELECT * FROM home_games h, away_games a
WHERE h.Game_Num_Home = a.Game_Num_Away

 * sqlite:///basketball.sqlite
Done.
Returning data to local variable team_stats_1610612765


In [22]:
%%sql team_stats_1610612747 << WITH home_games AS ( SELECT ROW_NUMBER() OVER(ORDER BY GAME_DATE) AS Game_Num_Home,
        TEAM_ID_HOME AS TEAM_ID, TEAM_ABBREVIATION_HOME AS TEAM_ABBREVIATION,
        AVG(FGM_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGM_HOME_avg,
        AVG(FGA_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGA_HOME_avg,
        AVG(FG3M_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_HOME_avg,
        AVG(FG3M_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_HOME_avg,
        AVG(FTM_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTM_HOME_avg,
        AVG(FTA_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTA_HOME_avg,
        AVG(OREB_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS OREB_HOME_avg,
        AVG(DREB_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS DREB_HOME_avg,
        AVG(AST_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS AST_HOME_avg,
        AVG(STL_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS STL_HOME_avg,
        AVG(TOV_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS TOV_HOME_avg,
        AVG(PF_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PF_HOME_avg,
        AVG(PTS_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PTS_HOME_avg,
        CASE
            WHEN WL_HOME = 'W'
            THEN 1
            ELSE 0
        END AS home_win
FROM Game
WHERE Season >= {season_start} AND TEAM_ID_HOME = 1610612747
),
away_games AS (
    SELECT ROW_NUMBER() OVER(ORDER BY GAME_DATE) AS Game_Num_Away,
        AVG(FGM_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGM_AWAY_avg,
        AVG(FGA_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGA_AWAY_avg,
        AVG(FG3M_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_AWAY_avg,
        AVG(FG3M_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_AWAY_avg,
        AVG(FTM_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTM_AWAY_avg,
        AVG(FTA_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTA_AWAY_avg,
        AVG(OREB_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS OREB_AWAY_avg,
        AVG(DREB_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS DREB_AWAY_avg,
        AVG(AST_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS AST_AWAY_avg,
        AVG(STL_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS STL_AWAY_avg,
        AVG(TOV_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS TOV_AWAY_avg,
        AVG(PF_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PF_AWAY_avg,
        AVG(PTS_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PTS_AWAY_avg,
        CASE
            WHEN WL_AWAY = 'W'
            THEN 1
            ELSE 0
        END AS away_win
    FROM Game
    WHERE Season >= {season_start} AND TEAM_ID_AWAY = 1610612747
)
SELECT * FROM home_games h, away_games a
WHERE h.Game_Num_Home = a.Game_Num_Away

 * sqlite:///basketball.sqlite
Done.
Returning data to local variable team_stats_1610612747


In [23]:
%%sql team_stats_1610612738 << WITH home_games AS ( SELECT ROW_NUMBER() OVER(ORDER BY GAME_DATE) AS Game_Num_Home,
        TEAM_ID_HOME AS TEAM_ID, TEAM_ABBREVIATION_HOME AS TEAM_ABBREVIATION,
        AVG(FGM_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGM_HOME_avg,
        AVG(FGA_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGA_HOME_avg,
        AVG(FG3M_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_HOME_avg,
        AVG(FG3M_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_HOME_avg,
        AVG(FTM_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTM_HOME_avg,
        AVG(FTA_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTA_HOME_avg,
        AVG(OREB_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS OREB_HOME_avg,
        AVG(DREB_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS DREB_HOME_avg,
        AVG(AST_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS AST_HOME_avg,
        AVG(STL_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS STL_HOME_avg,
        AVG(TOV_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS TOV_HOME_avg,
        AVG(PF_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PF_HOME_avg,
        AVG(PTS_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PTS_HOME_avg,
        CASE
            WHEN WL_HOME = 'W'
            THEN 1
            ELSE 0
        END AS home_win
FROM Game
WHERE Season >= {season_start} AND TEAM_ID_HOME = 1610612738
),
away_games AS (
    SELECT ROW_NUMBER() OVER(ORDER BY GAME_DATE) AS Game_Num_Away,
        AVG(FGM_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGM_AWAY_avg,
        AVG(FGA_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGA_AWAY_avg,
        AVG(FG3M_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_AWAY_avg,
        AVG(FG3M_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_AWAY_avg,
        AVG(FTM_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTM_AWAY_avg,
        AVG(FTA_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTA_AWAY_avg,
        AVG(OREB_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS OREB_AWAY_avg,
        AVG(DREB_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS DREB_AWAY_avg,
        AVG(AST_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS AST_AWAY_avg,
        AVG(STL_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS STL_AWAY_avg,
        AVG(TOV_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS TOV_AWAY_avg,
        AVG(PF_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PF_AWAY_avg,
        AVG(PTS_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PTS_AWAY_avg,
        CASE
            WHEN WL_AWAY = 'W'
            THEN 1
            ELSE 0
        END AS away_win
    FROM Game
    WHERE Season >= {season_start} AND TEAM_ID_AWAY = 1610612738
)
SELECT * FROM home_games h, away_games a
WHERE h.Game_Num_Home = a.Game_Num_Away

 * sqlite:///basketball.sqlite
Done.
Returning data to local variable team_stats_1610612738


In [24]:
%%sql team_stats_1610612743 << WITH home_games AS ( SELECT ROW_NUMBER() OVER(ORDER BY GAME_DATE) AS Game_Num_Home,
        TEAM_ID_HOME AS TEAM_ID, TEAM_ABBREVIATION_HOME AS TEAM_ABBREVIATION,
        AVG(FGM_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGM_HOME_avg,
        AVG(FGA_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGA_HOME_avg,
        AVG(FG3M_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_HOME_avg,
        AVG(FG3M_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_HOME_avg,
        AVG(FTM_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTM_HOME_avg,
        AVG(FTA_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTA_HOME_avg,
        AVG(OREB_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS OREB_HOME_avg,
        AVG(DREB_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS DREB_HOME_avg,
        AVG(AST_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS AST_HOME_avg,
        AVG(STL_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS STL_HOME_avg,
        AVG(TOV_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS TOV_HOME_avg,
        AVG(PF_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PF_HOME_avg,
        AVG(PTS_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PTS_HOME_avg,
        CASE
            WHEN WL_HOME = 'W'
            THEN 1
            ELSE 0
        END AS home_win
FROM Game
WHERE Season >= {season_start} AND TEAM_ID_HOME = 1610612743
),
away_games AS (
    SELECT ROW_NUMBER() OVER(ORDER BY GAME_DATE) AS Game_Num_Away,
        AVG(FGM_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGM_AWAY_avg,
        AVG(FGA_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGA_AWAY_avg,
        AVG(FG3M_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_AWAY_avg,
        AVG(FG3M_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_AWAY_avg,
        AVG(FTM_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTM_AWAY_avg,
        AVG(FTA_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTA_AWAY_avg,
        AVG(OREB_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS OREB_AWAY_avg,
        AVG(DREB_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS DREB_AWAY_avg,
        AVG(AST_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS AST_AWAY_avg,
        AVG(STL_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS STL_AWAY_avg,
        AVG(TOV_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS TOV_AWAY_avg,
        AVG(PF_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PF_AWAY_avg,
        AVG(PTS_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PTS_AWAY_avg,
        CASE
            WHEN WL_AWAY = 'W'
            THEN 1
            ELSE 0
        END AS away_win
    FROM Game
    WHERE Season >= {season_start} AND TEAM_ID_AWAY = 1610612743
)
SELECT * FROM home_games h, away_games a
WHERE h.Game_Num_Home = a.Game_Num_Away

 * sqlite:///basketball.sqlite
Done.
Returning data to local variable team_stats_1610612743


In [25]:
%%sql team_stats_1610612760 << WITH home_games AS ( SELECT ROW_NUMBER() OVER(ORDER BY GAME_DATE) AS Game_Num_Home,
        TEAM_ID_HOME AS TEAM_ID, TEAM_ABBREVIATION_HOME AS TEAM_ABBREVIATION,
        AVG(FGM_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGM_HOME_avg,
        AVG(FGA_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGA_HOME_avg,
        AVG(FG3M_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_HOME_avg,
        AVG(FG3M_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_HOME_avg,
        AVG(FTM_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTM_HOME_avg,
        AVG(FTA_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTA_HOME_avg,
        AVG(OREB_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS OREB_HOME_avg,
        AVG(DREB_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS DREB_HOME_avg,
        AVG(AST_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS AST_HOME_avg,
        AVG(STL_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS STL_HOME_avg,
        AVG(TOV_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS TOV_HOME_avg,
        AVG(PF_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PF_HOME_avg,
        AVG(PTS_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PTS_HOME_avg,
        CASE
            WHEN WL_HOME = 'W'
            THEN 1
            ELSE 0
        END AS home_win
FROM Game
WHERE Season >= {season_start} AND TEAM_ID_HOME = 1610612760
),
away_games AS (
    SELECT ROW_NUMBER() OVER(ORDER BY GAME_DATE) AS Game_Num_Away,
        AVG(FGM_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGM_AWAY_avg,
        AVG(FGA_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGA_AWAY_avg,
        AVG(FG3M_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_AWAY_avg,
        AVG(FG3M_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_AWAY_avg,
        AVG(FTM_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTM_AWAY_avg,
        AVG(FTA_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTA_AWAY_avg,
        AVG(OREB_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS OREB_AWAY_avg,
        AVG(DREB_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS DREB_AWAY_avg,
        AVG(AST_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS AST_AWAY_avg,
        AVG(STL_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS STL_AWAY_avg,
        AVG(TOV_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS TOV_AWAY_avg,
        AVG(PF_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PF_AWAY_avg,
        AVG(PTS_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PTS_AWAY_avg,
        CASE
            WHEN WL_AWAY = 'W'
            THEN 1
            ELSE 0
        END AS away_win
    FROM Game
    WHERE Season >= {season_start} AND TEAM_ID_AWAY = 1610612760
)
SELECT * FROM home_games h, away_games a
WHERE h.Game_Num_Home = a.Game_Num_Away

 * sqlite:///basketball.sqlite
Done.
Returning data to local variable team_stats_1610612760


In [26]:
%%sql team_stats_1610612764 << WITH home_games AS ( SELECT ROW_NUMBER() OVER(ORDER BY GAME_DATE) AS Game_Num_Home,
        TEAM_ID_HOME AS TEAM_ID, TEAM_ABBREVIATION_HOME AS TEAM_ABBREVIATION,
        AVG(FGM_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGM_HOME_avg,
        AVG(FGA_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGA_HOME_avg,
        AVG(FG3M_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_HOME_avg,
        AVG(FG3M_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_HOME_avg,
        AVG(FTM_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTM_HOME_avg,
        AVG(FTA_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTA_HOME_avg,
        AVG(OREB_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS OREB_HOME_avg,
        AVG(DREB_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS DREB_HOME_avg,
        AVG(AST_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS AST_HOME_avg,
        AVG(STL_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS STL_HOME_avg,
        AVG(TOV_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS TOV_HOME_avg,
        AVG(PF_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PF_HOME_avg,
        AVG(PTS_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PTS_HOME_avg,
        CASE
            WHEN WL_HOME = 'W'
            THEN 1
            ELSE 0
        END AS home_win
FROM Game
WHERE Season >= {season_start} AND TEAM_ID_HOME = 1610612764
),
away_games AS (
    SELECT ROW_NUMBER() OVER(ORDER BY GAME_DATE) AS Game_Num_Away,
        AVG(FGM_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGM_AWAY_avg,
        AVG(FGA_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGA_AWAY_avg,
        AVG(FG3M_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_AWAY_avg,
        AVG(FG3M_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_AWAY_avg,
        AVG(FTM_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTM_AWAY_avg,
        AVG(FTA_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTA_AWAY_avg,
        AVG(OREB_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS OREB_AWAY_avg,
        AVG(DREB_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS DREB_AWAY_avg,
        AVG(AST_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS AST_AWAY_avg,
        AVG(STL_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS STL_AWAY_avg,
        AVG(TOV_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS TOV_AWAY_avg,
        AVG(PF_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PF_AWAY_avg,
        AVG(PTS_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PTS_AWAY_avg,
        CASE
            WHEN WL_AWAY = 'W'
            THEN 1
            ELSE 0
        END AS away_win
    FROM Game
    WHERE Season >= {season_start} AND TEAM_ID_AWAY = 1610612764
)
SELECT * FROM home_games h, away_games a
WHERE h.Game_Num_Home = a.Game_Num_Away

 * sqlite:///basketball.sqlite
Done.
Returning data to local variable team_stats_1610612764


In [27]:
%%sql team_stats_1610612758 << WITH home_games AS ( SELECT ROW_NUMBER() OVER(ORDER BY GAME_DATE) AS Game_Num_Home,
        TEAM_ID_HOME AS TEAM_ID, TEAM_ABBREVIATION_HOME AS TEAM_ABBREVIATION,
        AVG(FGM_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGM_HOME_avg,
        AVG(FGA_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGA_HOME_avg,
        AVG(FG3M_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_HOME_avg,
        AVG(FG3M_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_HOME_avg,
        AVG(FTM_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTM_HOME_avg,
        AVG(FTA_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTA_HOME_avg,
        AVG(OREB_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS OREB_HOME_avg,
        AVG(DREB_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS DREB_HOME_avg,
        AVG(AST_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS AST_HOME_avg,
        AVG(STL_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS STL_HOME_avg,
        AVG(TOV_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS TOV_HOME_avg,
        AVG(PF_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PF_HOME_avg,
        AVG(PTS_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PTS_HOME_avg,
        CASE
            WHEN WL_HOME = 'W'
            THEN 1
            ELSE 0
        END AS home_win
FROM Game
WHERE Season >= {season_start} AND TEAM_ID_HOME = 1610612758
),
away_games AS (
    SELECT ROW_NUMBER() OVER(ORDER BY GAME_DATE) AS Game_Num_Away,
        AVG(FGM_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGM_AWAY_avg,
        AVG(FGA_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGA_AWAY_avg,
        AVG(FG3M_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_AWAY_avg,
        AVG(FG3M_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_AWAY_avg,
        AVG(FTM_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTM_AWAY_avg,
        AVG(FTA_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTA_AWAY_avg,
        AVG(OREB_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS OREB_AWAY_avg,
        AVG(DREB_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS DREB_AWAY_avg,
        AVG(AST_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS AST_AWAY_avg,
        AVG(STL_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS STL_AWAY_avg,
        AVG(TOV_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS TOV_AWAY_avg,
        AVG(PF_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PF_AWAY_avg,
        AVG(PTS_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PTS_AWAY_avg,
        CASE
            WHEN WL_AWAY = 'W'
            THEN 1
            ELSE 0
        END AS away_win
    FROM Game
    WHERE Season >= {season_start} AND TEAM_ID_AWAY = 1610612758
)
SELECT * FROM home_games h, away_games a
WHERE h.Game_Num_Home = a.Game_Num_Away

 * sqlite:///basketball.sqlite
Done.
Returning data to local variable team_stats_1610612758


In [28]:
%%sql team_stats_1610612763 << WITH home_games AS ( SELECT ROW_NUMBER() OVER(ORDER BY GAME_DATE) AS Game_Num_Home,
        TEAM_ID_HOME AS TEAM_ID, TEAM_ABBREVIATION_HOME AS TEAM_ABBREVIATION,
        AVG(FGM_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGM_HOME_avg,
        AVG(FGA_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGA_HOME_avg,
        AVG(FG3M_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_HOME_avg,
        AVG(FG3M_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_HOME_avg,
        AVG(FTM_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTM_HOME_avg,
        AVG(FTA_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTA_HOME_avg,
        AVG(OREB_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS OREB_HOME_avg,
        AVG(DREB_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS DREB_HOME_avg,
        AVG(AST_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS AST_HOME_avg,
        AVG(STL_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS STL_HOME_avg,
        AVG(TOV_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS TOV_HOME_avg,
        AVG(PF_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PF_HOME_avg,
        AVG(PTS_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PTS_HOME_avg,
        CASE
            WHEN WL_HOME = 'W'
            THEN 1
            ELSE 0
        END AS home_win
FROM Game
WHERE Season >= {season_start} AND TEAM_ID_HOME = 1610612763
),
away_games AS (
    SELECT ROW_NUMBER() OVER(ORDER BY GAME_DATE) AS Game_Num_Away,
        AVG(FGM_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGM_AWAY_avg,
        AVG(FGA_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGA_AWAY_avg,
        AVG(FG3M_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_AWAY_avg,
        AVG(FG3M_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_AWAY_avg,
        AVG(FTM_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTM_AWAY_avg,
        AVG(FTA_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTA_AWAY_avg,
        AVG(OREB_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS OREB_AWAY_avg,
        AVG(DREB_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS DREB_AWAY_avg,
        AVG(AST_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS AST_AWAY_avg,
        AVG(STL_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS STL_AWAY_avg,
        AVG(TOV_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS TOV_AWAY_avg,
        AVG(PF_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PF_AWAY_avg,
        AVG(PTS_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PTS_AWAY_avg,
        CASE
            WHEN WL_AWAY = 'W'
            THEN 1
            ELSE 0
        END AS away_win
    FROM Game
    WHERE Season >= {season_start} AND TEAM_ID_AWAY = 1610612763
)
SELECT * FROM home_games h, away_games a
WHERE h.Game_Num_Home = a.Game_Num_Away

 * sqlite:///basketball.sqlite
Done.
Returning data to local variable team_stats_1610612763


In [29]:
%%sql team_stats_1610612740 << WITH home_games AS ( SELECT ROW_NUMBER() OVER(ORDER BY GAME_DATE) AS Game_Num_Home,
        TEAM_ID_HOME AS TEAM_ID, TEAM_ABBREVIATION_HOME AS TEAM_ABBREVIATION,
        AVG(FGM_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGM_HOME_avg,
        AVG(FGA_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGA_HOME_avg,
        AVG(FG3M_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_HOME_avg,
        AVG(FG3M_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_HOME_avg,
        AVG(FTM_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTM_HOME_avg,
        AVG(FTA_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTA_HOME_avg,
        AVG(OREB_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS OREB_HOME_avg,
        AVG(DREB_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS DREB_HOME_avg,
        AVG(AST_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS AST_HOME_avg,
        AVG(STL_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS STL_HOME_avg,
        AVG(TOV_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS TOV_HOME_avg,
        AVG(PF_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PF_HOME_avg,
        AVG(PTS_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PTS_HOME_avg,
        CASE
            WHEN WL_HOME = 'W'
            THEN 1
            ELSE 0
        END AS home_win
FROM Game
WHERE Season >= {season_start} AND TEAM_ID_HOME = 1610612740
),
away_games AS (
    SELECT ROW_NUMBER() OVER(ORDER BY GAME_DATE) AS Game_Num_Away,
        AVG(FGM_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGM_AWAY_avg,
        AVG(FGA_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGA_AWAY_avg,
        AVG(FG3M_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_AWAY_avg,
        AVG(FG3M_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_AWAY_avg,
        AVG(FTM_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTM_AWAY_avg,
        AVG(FTA_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTA_AWAY_avg,
        AVG(OREB_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS OREB_AWAY_avg,
        AVG(DREB_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS DREB_AWAY_avg,
        AVG(AST_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS AST_AWAY_avg,
        AVG(STL_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS STL_AWAY_avg,
        AVG(TOV_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS TOV_AWAY_avg,
        AVG(PF_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PF_AWAY_avg,
        AVG(PTS_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PTS_AWAY_avg,
        CASE
            WHEN WL_AWAY = 'W'
            THEN 1
            ELSE 0
        END AS away_win
    FROM Game
    WHERE Season >= {season_start} AND TEAM_ID_AWAY = 1610612740
)
SELECT * FROM home_games h, away_games a
WHERE h.Game_Num_Home = a.Game_Num_Away

 * sqlite:///basketball.sqlite
Done.
Returning data to local variable team_stats_1610612740


In [30]:
%%sql team_stats_1610612741 << WITH home_games AS ( SELECT ROW_NUMBER() OVER(ORDER BY GAME_DATE) AS Game_Num_Home,
        TEAM_ID_HOME AS TEAM_ID, TEAM_ABBREVIATION_HOME AS TEAM_ABBREVIATION,
        AVG(FGM_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGM_HOME_avg,
        AVG(FGA_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGA_HOME_avg,
        AVG(FG3M_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_HOME_avg,
        AVG(FG3M_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_HOME_avg,
        AVG(FTM_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTM_HOME_avg,
        AVG(FTA_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTA_HOME_avg,
        AVG(OREB_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS OREB_HOME_avg,
        AVG(DREB_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS DREB_HOME_avg,
        AVG(AST_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS AST_HOME_avg,
        AVG(STL_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS STL_HOME_avg,
        AVG(TOV_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS TOV_HOME_avg,
        AVG(PF_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PF_HOME_avg,
        AVG(PTS_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PTS_HOME_avg,
        CASE
            WHEN WL_HOME = 'W'
            THEN 1
            ELSE 0
        END AS home_win
FROM Game
WHERE Season >= {season_start} AND TEAM_ID_HOME = 1610612741
),
away_games AS (
    SELECT ROW_NUMBER() OVER(ORDER BY GAME_DATE) AS Game_Num_Away,
        AVG(FGM_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGM_AWAY_avg,
        AVG(FGA_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGA_AWAY_avg,
        AVG(FG3M_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_AWAY_avg,
        AVG(FG3M_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_AWAY_avg,
        AVG(FTM_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTM_AWAY_avg,
        AVG(FTA_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTA_AWAY_avg,
        AVG(OREB_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS OREB_AWAY_avg,
        AVG(DREB_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS DREB_AWAY_avg,
        AVG(AST_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS AST_AWAY_avg,
        AVG(STL_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS STL_AWAY_avg,
        AVG(TOV_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS TOV_AWAY_avg,
        AVG(PF_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PF_AWAY_avg,
        AVG(PTS_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PTS_AWAY_avg,
        CASE
            WHEN WL_AWAY = 'W'
            THEN 1
            ELSE 0
        END AS away_win
    FROM Game
    WHERE Season >= {season_start} AND TEAM_ID_AWAY = 1610612741
)
SELECT * FROM home_games h, away_games a
WHERE h.Game_Num_Home = a.Game_Num_Away

 * sqlite:///basketball.sqlite
Done.
Returning data to local variable team_stats_1610612741


In [31]:
%%sql team_stats_1610612752 << WITH home_games AS ( SELECT ROW_NUMBER() OVER(ORDER BY GAME_DATE) AS Game_Num_Home,
        TEAM_ID_HOME AS TEAM_ID, TEAM_ABBREVIATION_HOME AS TEAM_ABBREVIATION,
        AVG(FGM_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGM_HOME_avg,
        AVG(FGA_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGA_HOME_avg,
        AVG(FG3M_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_HOME_avg,
        AVG(FG3M_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_HOME_avg,
        AVG(FTM_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTM_HOME_avg,
        AVG(FTA_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTA_HOME_avg,
        AVG(OREB_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS OREB_HOME_avg,
        AVG(DREB_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS DREB_HOME_avg,
        AVG(AST_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS AST_HOME_avg,
        AVG(STL_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS STL_HOME_avg,
        AVG(TOV_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS TOV_HOME_avg,
        AVG(PF_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PF_HOME_avg,
        AVG(PTS_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PTS_HOME_avg,
        CASE
            WHEN WL_HOME = 'W'
            THEN 1
            ELSE 0
        END AS home_win
FROM Game
WHERE Season >= {season_start} AND TEAM_ID_HOME = 1610612752
),
away_games AS (
    SELECT ROW_NUMBER() OVER(ORDER BY GAME_DATE) AS Game_Num_Away,
        AVG(FGM_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGM_AWAY_avg,
        AVG(FGA_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGA_AWAY_avg,
        AVG(FG3M_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_AWAY_avg,
        AVG(FG3M_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_AWAY_avg,
        AVG(FTM_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTM_AWAY_avg,
        AVG(FTA_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTA_AWAY_avg,
        AVG(OREB_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS OREB_AWAY_avg,
        AVG(DREB_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS DREB_AWAY_avg,
        AVG(AST_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS AST_AWAY_avg,
        AVG(STL_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS STL_AWAY_avg,
        AVG(TOV_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS TOV_AWAY_avg,
        AVG(PF_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PF_AWAY_avg,
        AVG(PTS_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PTS_AWAY_avg,
        CASE
            WHEN WL_AWAY = 'W'
            THEN 1
            ELSE 0
        END AS away_win
    FROM Game
    WHERE Season >= {season_start} AND TEAM_ID_AWAY = 1610612752
)
SELECT * FROM home_games h, away_games a
WHERE h.Game_Num_Home = a.Game_Num_Away

 * sqlite:///basketball.sqlite
Done.
Returning data to local variable team_stats_1610612752


In [32]:
%%sql team_stats_1610612739 << WITH home_games AS ( SELECT ROW_NUMBER() OVER(ORDER BY GAME_DATE) AS Game_Num_Home,
        TEAM_ID_HOME AS TEAM_ID, TEAM_ABBREVIATION_HOME AS TEAM_ABBREVIATION,
        AVG(FGM_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGM_HOME_avg,
        AVG(FGA_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGA_HOME_avg,
        AVG(FG3M_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_HOME_avg,
        AVG(FG3M_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_HOME_avg,
        AVG(FTM_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTM_HOME_avg,
        AVG(FTA_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTA_HOME_avg,
        AVG(OREB_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS OREB_HOME_avg,
        AVG(DREB_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS DREB_HOME_avg,
        AVG(AST_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS AST_HOME_avg,
        AVG(STL_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS STL_HOME_avg,
        AVG(TOV_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS TOV_HOME_avg,
        AVG(PF_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PF_HOME_avg,
        AVG(PTS_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PTS_HOME_avg,
        CASE
            WHEN WL_HOME = 'W'
            THEN 1
            ELSE 0
        END AS home_win
FROM Game
WHERE Season >= {season_start} AND TEAM_ID_HOME = 1610612739
),
away_games AS (
    SELECT ROW_NUMBER() OVER(ORDER BY GAME_DATE) AS Game_Num_Away,
        AVG(FGM_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGM_AWAY_avg,
        AVG(FGA_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGA_AWAY_avg,
        AVG(FG3M_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_AWAY_avg,
        AVG(FG3M_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_AWAY_avg,
        AVG(FTM_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTM_AWAY_avg,
        AVG(FTA_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTA_AWAY_avg,
        AVG(OREB_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS OREB_AWAY_avg,
        AVG(DREB_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS DREB_AWAY_avg,
        AVG(AST_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS AST_AWAY_avg,
        AVG(STL_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS STL_AWAY_avg,
        AVG(TOV_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS TOV_AWAY_avg,
        AVG(PF_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PF_AWAY_avg,
        AVG(PTS_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PTS_AWAY_avg,
        CASE
            WHEN WL_AWAY = 'W'
            THEN 1
            ELSE 0
        END AS away_win
    FROM Game
    WHERE Season >= {season_start} AND TEAM_ID_AWAY = 1610612739
)
SELECT * FROM home_games h, away_games a
WHERE h.Game_Num_Home = a.Game_Num_Away

 * sqlite:///basketball.sqlite
Done.
Returning data to local variable team_stats_1610612739


In [33]:
%%sql team_stats_1610612749 << WITH home_games AS ( SELECT ROW_NUMBER() OVER(ORDER BY GAME_DATE) AS Game_Num_Home,
        TEAM_ID_HOME AS TEAM_ID, TEAM_ABBREVIATION_HOME AS TEAM_ABBREVIATION,
        AVG(FGM_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGM_HOME_avg,
        AVG(FGA_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGA_HOME_avg,
        AVG(FG3M_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_HOME_avg,
        AVG(FG3M_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_HOME_avg,
        AVG(FTM_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTM_HOME_avg,
        AVG(FTA_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTA_HOME_avg,
        AVG(OREB_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS OREB_HOME_avg,
        AVG(DREB_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS DREB_HOME_avg,
        AVG(AST_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS AST_HOME_avg,
        AVG(STL_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS STL_HOME_avg,
        AVG(TOV_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS TOV_HOME_avg,
        AVG(PF_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PF_HOME_avg,
        AVG(PTS_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PTS_HOME_avg,
        CASE
            WHEN WL_HOME = 'W'
            THEN 1
            ELSE 0
        END AS home_win
FROM Game
WHERE Season >= {season_start} AND TEAM_ID_HOME = 1610612749
),
away_games AS (
    SELECT ROW_NUMBER() OVER(ORDER BY GAME_DATE) AS Game_Num_Away,
        AVG(FGM_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGM_AWAY_avg,
        AVG(FGA_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGA_AWAY_avg,
        AVG(FG3M_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_AWAY_avg,
        AVG(FG3M_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_AWAY_avg,
        AVG(FTM_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTM_AWAY_avg,
        AVG(FTA_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTA_AWAY_avg,
        AVG(OREB_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS OREB_AWAY_avg,
        AVG(DREB_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS DREB_AWAY_avg,
        AVG(AST_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS AST_AWAY_avg,
        AVG(STL_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS STL_AWAY_avg,
        AVG(TOV_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS TOV_AWAY_avg,
        AVG(PF_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PF_AWAY_avg,
        AVG(PTS_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PTS_AWAY_avg,
        CASE
            WHEN WL_AWAY = 'W'
            THEN 1
            ELSE 0
        END AS away_win
    FROM Game
    WHERE Season >= {season_start} AND TEAM_ID_AWAY = 1610612749
)
SELECT * FROM home_games h, away_games a
WHERE h.Game_Num_Home = a.Game_Num_Away

 * sqlite:///basketball.sqlite
Done.
Returning data to local variable team_stats_1610612749


In [34]:
%%sql team_stats_1610612737 << WITH home_games AS ( SELECT ROW_NUMBER() OVER(ORDER BY GAME_DATE) AS Game_Num_Home,
        TEAM_ID_HOME AS TEAM_ID, TEAM_ABBREVIATION_HOME AS TEAM_ABBREVIATION,
        AVG(FGM_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGM_HOME_avg,
        AVG(FGA_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGA_HOME_avg,
        AVG(FG3M_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_HOME_avg,
        AVG(FG3M_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_HOME_avg,
        AVG(FTM_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTM_HOME_avg,
        AVG(FTA_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTA_HOME_avg,
        AVG(OREB_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS OREB_HOME_avg,
        AVG(DREB_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS DREB_HOME_avg,
        AVG(AST_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS AST_HOME_avg,
        AVG(STL_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS STL_HOME_avg,
        AVG(TOV_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS TOV_HOME_avg,
        AVG(PF_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PF_HOME_avg,
        AVG(PTS_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PTS_HOME_avg,
        CASE
            WHEN WL_HOME = 'W'
            THEN 1
            ELSE 0
        END AS home_win
FROM Game
WHERE Season >= {season_start} AND TEAM_ID_HOME = 1610612737
),
away_games AS (
    SELECT ROW_NUMBER() OVER(ORDER BY GAME_DATE) AS Game_Num_Away,
        AVG(FGM_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGM_AWAY_avg,
        AVG(FGA_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGA_AWAY_avg,
        AVG(FG3M_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_AWAY_avg,
        AVG(FG3M_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_AWAY_avg,
        AVG(FTM_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTM_AWAY_avg,
        AVG(FTA_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTA_AWAY_avg,
        AVG(OREB_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS OREB_AWAY_avg,
        AVG(DREB_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS DREB_AWAY_avg,
        AVG(AST_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS AST_AWAY_avg,
        AVG(STL_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS STL_AWAY_avg,
        AVG(TOV_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS TOV_AWAY_avg,
        AVG(PF_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PF_AWAY_avg,
        AVG(PTS_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PTS_AWAY_avg,
        CASE
            WHEN WL_AWAY = 'W'
            THEN 1
            ELSE 0
        END AS away_win
    FROM Game
    WHERE Season >= {season_start} AND TEAM_ID_AWAY = 1610612737
)
SELECT * FROM home_games h, away_games a
WHERE h.Game_Num_Home = a.Game_Num_Away

 * sqlite:///basketball.sqlite
Done.
Returning data to local variable team_stats_1610612737


In [35]:
%%sql team_stats_1610612750 << WITH home_games AS ( SELECT ROW_NUMBER() OVER(ORDER BY GAME_DATE) AS Game_Num_Home,
        TEAM_ID_HOME AS TEAM_ID, TEAM_ABBREVIATION_HOME AS TEAM_ABBREVIATION,
        AVG(FGM_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGM_HOME_avg,
        AVG(FGA_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGA_HOME_avg,
        AVG(FG3M_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_HOME_avg,
        AVG(FG3M_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_HOME_avg,
        AVG(FTM_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTM_HOME_avg,
        AVG(FTA_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTA_HOME_avg,
        AVG(OREB_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS OREB_HOME_avg,
        AVG(DREB_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS DREB_HOME_avg,
        AVG(AST_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS AST_HOME_avg,
        AVG(STL_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS STL_HOME_avg,
        AVG(TOV_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS TOV_HOME_avg,
        AVG(PF_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PF_HOME_avg,
        AVG(PTS_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PTS_HOME_avg,
        CASE
            WHEN WL_HOME = 'W'
            THEN 1
            ELSE 0
        END AS home_win
FROM Game
WHERE Season >= {season_start} AND TEAM_ID_HOME = 1610612750
),
away_games AS (
    SELECT ROW_NUMBER() OVER(ORDER BY GAME_DATE) AS Game_Num_Away,
        AVG(FGM_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGM_AWAY_avg,
        AVG(FGA_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGA_AWAY_avg,
        AVG(FG3M_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_AWAY_avg,
        AVG(FG3M_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_AWAY_avg,
        AVG(FTM_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTM_AWAY_avg,
        AVG(FTA_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTA_AWAY_avg,
        AVG(OREB_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS OREB_AWAY_avg,
        AVG(DREB_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS DREB_AWAY_avg,
        AVG(AST_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS AST_AWAY_avg,
        AVG(STL_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS STL_AWAY_avg,
        AVG(TOV_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS TOV_AWAY_avg,
        AVG(PF_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PF_AWAY_avg,
        AVG(PTS_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PTS_AWAY_avg,
        CASE
            WHEN WL_AWAY = 'W'
            THEN 1
            ELSE 0
        END AS away_win
    FROM Game
    WHERE Season >= {season_start} AND TEAM_ID_AWAY = 1610612750
)
SELECT * FROM home_games h, away_games a
WHERE h.Game_Num_Home = a.Game_Num_Away

 * sqlite:///basketball.sqlite
Done.
Returning data to local variable team_stats_1610612750


In [36]:
# Convert team result sets into dataframes
team_df_1610612750 = team_stats_1610612750.DataFrame()
team_df_1610612737 = team_stats_1610612737.DataFrame()
team_df_1610612749 = team_stats_1610612749.DataFrame()
team_df_1610612739 = team_stats_1610612739.DataFrame()
team_df_1610612752 = team_stats_1610612752.DataFrame()
team_df_1610612741 = team_stats_1610612741.DataFrame()
team_df_1610612740 = team_stats_1610612740.DataFrame()
team_df_1610612763 = team_stats_1610612763.DataFrame()
team_df_1610612758 = team_stats_1610612758.DataFrame()
team_df_1610612764 = team_stats_1610612764.DataFrame()
team_df_1610612760 = team_stats_1610612760.DataFrame()
team_df_1610612743 = team_stats_1610612743.DataFrame()
team_df_1610612738 = team_stats_1610612738.DataFrame()
team_df_1610612747 = team_stats_1610612747.DataFrame()
team_df_1610612765 = team_stats_1610612765.DataFrame()
team_df_1610612745 = team_stats_1610612745.DataFrame()
team_df_1610612744 = team_stats_1610612744.DataFrame()
team_df_1610612766 = team_stats_1610612766.DataFrame()
team_df_1610612742 = team_stats_1610612742.DataFrame()
team_df_1610612754 = team_stats_1610612754.DataFrame()
team_df_1610612762 = team_stats_1610612762.DataFrame()
team_df_1610612757 = team_stats_1610612757.DataFrame()
team_df_1610612748 = team_stats_1610612748.DataFrame()
team_df_1610612753 = team_stats_1610612753.DataFrame()
team_df_1610612751 = team_stats_1610612751.DataFrame()
team_df_1610612756 = team_stats_1610612756.DataFrame()
team_df_1610612759 = team_stats_1610612759.DataFrame()
team_df_1610612755 = team_stats_1610612755.DataFrame()
team_df_1610612761 = team_stats_1610612761.DataFrame()
team_df_1610612746 = team_stats_1610612746.DataFrame()

In [37]:
# Union the game data for all the teams
comb_df = pd.concat([team_df_1610612750, team_df_1610612737, team_df_1610612749, team_df_1610612739, team_df_1610612752, 
          team_df_1610612741, team_df_1610612740, team_df_1610612763, team_df_1610612758, team_df_1610612764, 
         team_df_1610612760, team_df_1610612743, team_df_1610612738, team_df_1610612747, team_df_1610612765, 
         team_df_1610612745, team_df_1610612744, team_df_1610612766, team_df_1610612742, team_df_1610612754, 
         team_df_1610612762, team_df_1610612757, team_df_1610612748, team_df_1610612753, team_df_1610612751, 
         team_df_1610612756, team_df_1610612759, team_df_1610612755, team_df_1610612761, team_df_1610612746])
comb_df.head()

,Game_Num_Home,TEAM_ID,TEAM_ABBREVIATION,FGM_HOME_avg,FGA_HOME_avg,FG3M_HOME_avg,FG3M_HOME_avg:1,FTM_HOME_avg,FTA_HOME_avg,OREB_HOME_avg,...,FTM_AWAY_avg,FTA_AWAY_avg,OREB_AWAY_avg,DREB_AWAY_avg,AST_AWAY_avg,STL_AWAY_avg,TOV_AWAY_avg,PF_AWAY_avg,PTS_AWAY_avg,away_win
0,1,1610612750,MIN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,2,1610612750,MIN,44.000000,87.000000,10.00,10.00,33.000000,35.000000,9.000000,...,24.000000,27.000000,14.0,32.0,20.000000,9.000000,11.000000,27.000000,108.000000,0
2,3,1610612750,MIN,40.000000,86.000000,8.50,8.50,27.500000,29.500000,8.500000,...,29.000000,33.500000,10.0,29.5,21.500000,9.500000,14.000000,28.500000,122.000000,0
3,4,1610612750,MIN,38.666667,94.666667,9.00,9.00,22.666667,25.666667,11.333333,...,22.666667,27.666667,12.0,29.0,22.666667,9.666667,14.666667,24.666667,116.333333,0
4,5,1610612750,MIN,39.500000,95.500000,10.25,10.25,23.500000,26.750000,13.500000,...,19.500000,24.250000,11.0,29.5,23.250000,9.250000,12.250000,22.500000,112.000000,0


In [38]:
comb_df['home_win'].head()

0    1
1    1
2    0
3    1
4    1
Name: home_win, dtype: int64

In [39]:
comb_df.shape

(3347, 32)